
<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

# <center>Getting Started with Tracing

This guide demonstrates how to use Arize for monitoring and debugging your LLM using Traces and Spans. You can read more about LLM tracing [here](https://docs.arize.com/arize/llm-large-language-models/llm-traces). In this tutorial, you will use opentelemetry and [openinference](https://github.com/Arize-ai/openinference/tree/main) to instrument our application in order to send traces to Arize.

ℹ️ This notebook requires:
- An OpenAI API key
- An Arize Space & API Key


## Step 1: Install Dependencies 📚
Let's get the notebook setup with dependencies.

In [ ]:
# Dependencies needed to instrument your openai application using opentelemetry and openinference
!pip install arize-otel openai openinference-instrumentation-openai opentelemetry-sdk opentelemetry-exporter-otlp

INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-instrumentation to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.17.0 requires protobuf!=4.24.0,<5.0.0,>=3.19.6, but you have protobuf 5.28.3 which is incompatible.
tensorflow 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3

## Step 2: Get your API Keys

Copy the Arize API_KEY and SPACE_ID from your Space Settings page (shown below) to the variables in the cell below.

<center><img src="https://storage.googleapis.com/arize-assets/barcelos/Screenshot%202024-11-11%20at%209.28.27%E2%80%AFPM.png" width="700"></center>

In [ ]:
SPACE_ID = "SPACE_ID" # Change this line
API_KEY = "API_KEY" # Change this line

if SPACE_ID == "SPACE_ID" or API_KEY == "API_KEY":
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")
else:
    print("✅ Import and Setup Arize Client Done! Now we can start using Arize!")

✅ Import and Setup Arize Client Done! Now we can start using Arize!


Add your OpenAI key below to test OpenAI instrumentation

In [ ]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

🔑 Enter your OpenAI API key: ··········


## Step 3. Add our tracing code
We will be using the arize_otel package to register the URL and authentication parameters to send to Arize using OpenTelemetry. You can see what's under the hood by looking [here](https://docs.arize.com/arize/large-language-models/tracing/auto-instrumentation).

In [ ]:
# Import open-telemetry dependencies
from arize_otel import register_otel, Endpoints

# Setup OTEL via our convenience function
register_otel(
    endpoints = Endpoints.ARIZE,
    space_id = SPACE_ID, # in app space settings page
    api_key = API_KEY, # in app space settings page
    model_id = "tracing-haiku-tutorial", # name this to whatever you would like
)
# Import the automatic instrumentor from OpenInference
from openinference.instrumentation.openai import OpenAIInstrumentor

# Finish automatic instrumentation
OpenAIInstrumentor().instrument()

## Step 4: Run your LLM application
Let's test our app by asking OpenAI to write a haiku.

In [ ]:
import openai

# Run OpenAI completion
openai_client = openai.OpenAI()
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Write a haiku."}],
    max_tokens=20,
)
print(response.choices[0].message.content)

Whispering wind blows
Through the branches of tall trees
Nature's symphony.


Great! Our application works!

## Step 5: Log into Arize and explore your application traces 🚀

Log into your Arize account, and look for the model with the same `model_id`.

<img src="https://storage.googleapis.com/arize-assets/tutorials/images/arize-llm-tracing.png" width="1500" />